In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt

import src.workfile_functions as wf

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge, Ridge, RidgeCV, SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',100)
pd.options.display.float_format = '{:,.2f}'.format

PATH = '../data/pc/'

URL = "https://michael-weylandt.com/STA9890/competition_data/"

In [2]:
df = pd.read_csv('../data/pc/train.csv',index_col=None)
df.drop(['assessed_2019','building_value_2019','land_value_2019'],axis=1,inplace=True)

In [3]:
X = df.drop(columns=['acct','TARGET'])
y = df['TARGET']

In [4]:
test = pd.read_csv('../data/pc/test.csv',index_col=None)

In [5]:
X_test = test.drop_duplicates(subset='acct').drop(columns='acct',axis=1)

In [6]:
# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Correlation Testing

In [7]:
wf.get_high_correlations(X)[:10]

[('end_quality', 'end_quality_description', 1.0000000000036762),
 ('first_quality', 'first_quality_description', 0.9999999174712321),
 ('delta_quality', 'delta_quality_description', 0.9999998381680607),
 ('land_value_2016', 'land_value_2017', 0.9940747933465361),
 ('land_value_2017', 'land_value_2018', 0.9922369817413471),
 ('assessed_2016', 'assessed_2017', 0.9913824526848076),
 ('building_area_2017', 'building_area_2018', 0.9907776400540445),
 ('assessed_2017', 'assessed_2018', 0.9904900677183098),
 ('building_area_2016', 'building_area_2017', 0.9903148136216984),
 ('land_value_2015', 'land_value_2016', 0.9903092887703535)]

Lots of co-linearity, so $\ell_1$ might not be great to use

# Ridge CV with GridSearch and Validation

In [6]:
ridge = Ridge()
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

In [7]:
# Define scoring as negative RMSE
ridge_grid = GridSearchCV(
    ridge,
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error'  # gives negative RMSE
)

In [8]:
%%time
ridge_grid.fit(X_train, y_train)

Wall time: 1min 29s


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='neg_root_mean_squared_error')

In [9]:
# Best alpha
print("Best alpha:", ridge_grid.best_params_['alpha'])

Best alpha: 0.01


In [10]:
# Evaluate on validation set
val_preds = ridge_grid.predict(X_val)
rmse_val = mean_squared_error(y_val, val_preds, squared=False)
print("Validation RMSE:", rmse_val)

Validation RMSE: 48198.10234274999


# XGBoost Regressor

In [13]:
param_grid = {
    'n_estimators': [200],
    'max_depth': [3, 5],
    'learning_rate': [0.1],
    'subsample': [1.0]
}

In [14]:
xgb = XGBRegressor(objective='reg:squarederror', n_jobs=1, random_state=42)

In [15]:
xgb_grid = GridSearchCV(
    xgb,
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=8
)

In [16]:
%%time
xgb_grid.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=10,
    verbose=False
)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:  8.1min remaining:  5.4min
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed: 11.0min finished


Wall time: 16min 30s


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...imators=100, n_jobs=1,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=42, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
             

In [17]:
best_model = xgb_grid.best_estimator_

In [19]:
val_preds = best_model.predict(X_val)
val_rmse = mean_squared_error(y_val, val_preds, squared=False)

print("Best params:", xgb_grid.best_params_)
print("Validation RMSE:", val_rmse)

Best params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
Validation RMSE: 37582.50537174819


# LGBM Regressor

In [12]:
lgbm = LGBMRegressor(random_state=42)

In [13]:
param_grid_lgbm = {
    'n_estimators': [100, 300],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [31, 50],
    'max_depth': [-1, 5, 10]
}

In [14]:
grid_lgbm = GridSearchCV(
    lgbm,
    param_grid_lgbm,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=8,
    verbose=1
)

In [15]:
%%time
grid_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=10,
    verbose=False
)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  4.6min
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed: 10.5min finished


Wall time: 10min 39s


GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=42), n_jobs=8,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [-1, 5, 10],
                         'n_estimators': [100, 300], 'num_leaves': [31, 50]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [16]:
best_lgbm = grid_lgbm.best_estimator_

In [17]:
best_lgbm

LGBMRegressor(max_depth=10, n_estimators=300, random_state=42)

In [18]:
val_preds_lgbm = grid_lgbm.predict(X_val)

In [19]:
print("Best params", grid_lgbm.best_params_)
print("LGBM RMSE:", mean_squared_error(y_val, val_preds_lgbm, squared=False))

Best params {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 300, 'num_leaves': 31}
LGBM RMSE: 59590.531205763065


# LGBM 2

In [8]:
lgb = LGBMRegressor(
    random_state=42,
    n_estimators=250,
    learning_rate=0.05,
    num_leaves=15,
    max_depth=12,
    objective='regression'
)

In [9]:
%%time
lgb.fit(X_train, y_train)

Wall time: 5.88 s


LGBMRegressor(learning_rate=0.05, max_depth=12, n_estimators=250, num_leaves=15,
              objective='regression', random_state=42)

In [10]:
lgb_val_preds = lgb.predict(X_val)

In [11]:
print("LGBM RMSE:", mean_squared_error(y_val, lgb_val_preds, squared=False))

LGBM RMSE: 59025.3123927858


# SGD Regressor

In [6]:
sgd = SGDRegressor(random_state=42)

In [7]:
param_grid_sgd = {
    'alpha': [1e-4, 1e-3, 1e-2],               # regularization strength
    'penalty': ['l2', 'l1', 'elasticnet'],     # type of penalty
    'max_iter': [1000],
    'tol': [1e-3]
}


In [8]:
grid_sgd = GridSearchCV(
    estimator=sgd,
    param_grid=param_grid_sgd,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=4,
    verbose=1
)

In [9]:
%%time
grid_sgd.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:  1.8min finished


Wall time: 1min 53s


GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42), n_jobs=4,
             param_grid={'alpha': [0.0001, 0.001, 0.01], 'max_iter': [1000],
                         'penalty': ['l2', 'l1', 'elasticnet'],
                         'tol': [0.001]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [10]:
best_sgd = grid_sgd.best_estimator_
val_preds_sgd = best_sgd.predict(X_val)
val_rmse_sgd = mean_squared_error(y_val, val_preds_sgd, squared=False)

In [11]:
print("Best SGD Params:", grid_sgd.best_params_)
print("SGD RMSE:", val_rmse_sgd)

Best SGD Params: {'alpha': 0.001, 'max_iter': 1000, 'penalty': 'elasticnet', 'tol': 0.001}
SGD RMSE: 9.006211178924453e+24


# Bayesian Ridge Regressor

In [6]:
bayes = BayesianRidge()

param_grid_bayes = {
    'alpha_1': [1e-6, 1e-4],
    'alpha_2': [1e-6, 1e-4],
    'lambda_1': [1e-6, 1e-4],
    'lambda_2': [1e-6, 1e-4]
}

In [7]:
grid_bayes = GridSearchCV(
    bayes,
    param_grid_bayes,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=4,
    verbose=1
)

In [9]:
%%time
grid_bayes.fit(
    X_train, y_train
)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:  4.8min finished


Wall time: 4min 57s


GridSearchCV(cv=5, estimator=BayesianRidge(), n_jobs=4,
             param_grid={'alpha_1': [1e-06, 0.0001], 'alpha_2': [1e-06, 0.0001],
                         'lambda_1': [1e-06, 0.0001],
                         'lambda_2': [1e-06, 0.0001]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [10]:
best_bayes = grid_bayes.best_estimator_

In [11]:
val_preds_bayes = grid_bayes.predict(X_val)
print("Best Bayesian Ridge Params:", grid_bayes.best_params_)
print("Bayesian Ridge RMSE:", mean_squared_error(y_val, val_preds_bayes, squared=False))

Best Bayesian Ridge Params: {'alpha_1': 0.0001, 'alpha_2': 1e-06, 'lambda_1': 1e-06, 'lambda_2': 1e-06}
Bayesian Ridge RMSE: 47972.34967751687


# Random Forest Regressor

In [12]:
rf = RandomForestRegressor(
    random_state=42
)

In [ ]:
%%time
rf.fit(
    X_train,
    y_train
)

In [15]:
val_preds_rf = grid_rf.predict(X_val)

NameError: name 'grid_rf' is not defined

In [ ]:
print("Random Forest RMSE:", mean_squared_error(y_val, val_preds_rf, squared=False))

### GridSearch (takes a long time)

In [13]:
param_grid_rf = {
    'n_estimators': [100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

In [14]:
grid_rf = GridSearchCV(
    rf,
    param_grid_rf,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=4,
    verbose=1
)

In [ ]:
# %%time
# grid_rf.fit(
#     X_train, y_train
# )

In [ ]:
# best_rf = grid_rf.best_estimator_

In [ ]:
# val_preds_rf = grid_rf.predict(X_val)

In [ ]:
# print("Best Random Forest Params:", grid_rf.best_params_)
# print("Random Forest RMSE:", mean_squared_error(y_val, val_preds_rf, squared=False))

# Ensemble

## Model Retrain

First, re-train all models using full X

In [18]:
ridge = Ridge(alpha=0.01)
xgb = XGBRegressor(learning_rate=0.1, max_depth=5,
                  n_estimators=200,subsample=1.0)
bayes = BayesianRidge(alpha_1=0.0001, alpha_2=1e-06,
                     lambda_1 = 1e-06, lambda_2 = 1e-06)

In [21]:
%%time
ridge.fit(X,y)

Wall time: 4.2 s


Ridge(alpha=0.01)

In [22]:
%%time
xgb.fit(X,y)

Wall time: 53.8 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
%%time
bayes.fit(X,y)

Wall time: 7.64 s


BayesianRidge(alpha_1=0.0001)

## Predictions for Weights

In [27]:
%%time
pred_ridge = ridge.predict(X_test)

Wall time: 2.08 s


In [28]:
%%time
pred_xgb = xgb.predict(X_test)

Wall time: 2.15 s


In [29]:
%%time
pred_bayes = bayes.predict(X_test)

Wall time: 1.97 s


### Weighted Average

In [30]:
# use weights based on train MSE
mse = {
    'xgb':37582,
    'ridge':48198.1,
    'bayes':47972.35
}

In [31]:
inv_mse = {k: 1/v for k, v in mse.items()}

In [32]:
total = sum(inv_mse.values())

In [33]:
weights = {k: v/total for k, v in inv_mse.items()}

In [34]:
print(weights)

{'xgb': 0.39014495556855006, 'ridge': 0.3042117369808612, 'bayes': 0.30564330745058865}


## Create predictions

In [77]:
# ensure test is unique
test = test.drop_duplicates(subset='acct')

In [78]:
# create X_test
X_test = test.drop(columns=['acct'])

In [79]:
# get unique account ids
acct_ids = test['acct']

In [41]:
# get preds
ensemble_pred = (
    weights['xgb'] * xgb.predict(X_test) +
    weights['ridge'] * ridge.predict(X_test) +
    weights['bayes'] * bayes.predict(X_test)
)

In [105]:
val_ensemble_pred = (
    weights['xgb'] * xgb.predict(X_val) +
    weights['ridge'] * ridge.predict(X_val) +
    weights['bayes'] * bayes.predict(X_val)
)

In [106]:
ensemble_rmse = mean_squared_error(y_val, val_ensemble_pred, squared=False)
print("Weighted Ensemble RMSE on validation:", ensemble_rmse)

Weighted Ensemble RMSE on validation: 41068.0675619819


In [50]:
# create output dataframe
output_df = pd.DataFrame({
    'acct':acct_ids,
    'TARGET':ensemble_pred
})

In [51]:
# rename acct
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [52]:
# reset index
output_df.reset_index(inplace=True,drop=True)

In [53]:
output_df

,ACCOUNT,TARGET
0,bb75f25168addc1117840b10c0fd6cd0c2a7b7c6,"368,893.13"
1,8def0ccceda200b673872a8a9367644767989f3b,"140,048.70"
2,ca33e57b3b13e843909f4b6cbd9a3410387bd45a,"271,803.79"
3,3e0f6f6090a8226ce67ccf2f8630b8ad630b8d55,"164,018.02"
4,63facf82adbae10b23f7fabc93188c95bd832f51,"301,414.13"
...,...,...
418853,24847d36c333ab3376848ee1bda74916286a2a4b,"347,302.89"
418854,62ba19f1655097ac9cc8682aec435d22a653bfa0,"377,987.24"
418855,552b5300d14369d8fd792ea6228dfa683014b2f5,"444,930.59"
418856,290d351d16d00e79e89dc5404412f07769f40038,"301,386.46"


In [54]:
output_df.to_csv('../data/pc/predictions/gradient_descenters_ensemble_1.csv',index=None)

# Stacking

In [77]:
# ensure test is unique
test = test.drop_duplicates(subset='acct')

In [78]:
# create X_test
X_test = test.drop(columns=['acct'])

In [79]:
# get unique account ids
acct_ids = test['acct']

In [80]:
ridge = Ridge(alpha=0.01)
xgb = XGBRegressor(learning_rate=0.1, max_depth=5,
                  n_estimators=200,subsample=1.0)
bayes = BayesianRidge(alpha_1=0.0001, alpha_2=1e-06,
                     lambda_1 = 1e-06, lambda_2 = 1e-06)

In [81]:
%%time
ridge.fit(X_train,y_train)

Wall time: 3.59 s


Ridge(alpha=0.01)

In [82]:
%%time
xgb.fit(X_train,y_train)

Wall time: 43.5 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
%%time
bayes.fit(X_train,y_train)

Wall time: 5.68 s


BayesianRidge(alpha_1=0.0001)

In [84]:
val_preds_ridge = ridge.predict(X_val)
val_preds_xgb = xgb.predict(X_val)
val_preds_bayes = bayes.predict(X_val)

meta_X = np.column_stack([val_preds_ridge, val_preds_xgb, val_preds_bayes])
meta_y = y_val

In [100]:
meta_model = Ridge()
meta_model.fit(meta_X, meta_y)

Ridge()

In [101]:
test_meta_X = np.column_stack([
    ridge.predict(X_test),
    xgb.predict(X_test),
    bayes.predict(X_test)
])

final_preds = meta_model.predict(test_meta_X)

In [103]:
train_preds = meta_model.predict(meta_X)
train_rmse = mean_squared_error(meta_y, train_preds, squared=False)

print("Meta-Model Train RMSE:", train_rmse)

Meta-Model Train RMSE: 37393.20597984435


In [102]:
# create output dataframe
output_df = pd.DataFrame({
    'acct':acct_ids,
    'TARGET':final_preds
})

In [89]:
# rename acct
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [90]:
output_df.to_csv('../data/pc/predictions/gradient_descenters_stacking_1.csv',index=None)

### Trying with meta_model being XGBR

In [92]:
meta_model = XGBRegressor(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    subsample=1.0,
    random_state=42
)

In [93]:
meta_model.fit(meta_X, meta_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [94]:
test_meta_X = np.column_stack([
    ridge.predict(X_test),
    xgb.predict(X_test),
    bayes.predict(X_test)
])

final_preds = meta_model.predict(test_meta_X)

In [99]:
train_preds = meta_model.predict(meta_X)
train_rmse = mean_squared_error(meta_y, train_preds, squared=False)

print("Meta-Model Train RMSE:", train_rmse)

Meta-Model Train RMSE: 33441.118882366376


In [95]:
# create output dataframe
output_df = pd.DataFrame({
    'acct':acct_ids,
    'TARGET':final_preds
})

In [96]:
# rename acct
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [97]:
output_df.to_csv('../data/pc/predictions/gradient_descenters_stacking_2.csv',index=None)